# Image Alignment
---

In [1]:
from __future__ import print_function
from utils import *
from natsort import natsorted

import argparse
import cv2
import numpy as np

In [2]:
def compute_descriptors(imGray):
    """
    Calcula keypoints e descritores SIFT para uma imagem em tons de cinza.

    Args:
        imGray: Uma imagem em tons de cinza representada como um array NumPy.

    Returns:
        keypoints: Uma lista de keypoints detectados.
        descriptors: Um array NumPy contendo os descritores calculados.
    """

    # Cria um objeto SIFT para detecção de keypoints e cálculo de descritores
    sift = cv2.xfeatures2d.SIFT_create()

    # Detecta keypoints e calcula descritores usando SIFT
    keypoints, descriptors = sift.detectAndCompute(imGray, None)

    # Imprime o número de keypoints detectados e a forma do array de descritores
    print("keypoints: {}, descriptors: {}".format(len(keypoints), descriptors.shape))

    return keypoints, descriptors

In [3]:
def create_matcher(trees, checks):
    """
    Cria um objeto cv2.FlannBasedMatcher para correspondência de características.

    Args:
        trees: Número de árvores na estrutura de dados KD-Tree usada para busca rápida de vizinhos mais próximos.
        checks: Número de verificações realizadas durante a busca de correspondências. Aumentar esse valor melhora a precisão, mas também aumenta o tempo de processamento.

    Returns:
        matcher: Um objeto cv2.FlannBasedMatcher configurado para correspondência de características usando o algoritmo FLANN.
    """

    # Define o tipo de índice como KD-Tree
    FLANN_INDEX_KDTREE = 0

    # Parâmetros para a construção do índice KD-Tree
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=trees)

    # Parâmetros para a busca de correspondências
    search_params = dict(checks=checks)

    # Cria o objeto matcher usando os parâmetros definidos
    matcher = cv2.FlannBasedMatcher(index_params, search_params)

    return matcher

In [4]:
def find_good_matches_loc(matcher, keypoints1, descriptors1, keypoints2, descriptors2, factor):
    """
    Encontra correspondências de boa qualidade entre dois conjuntos de keypoints e descritores, e retorna suas localizações.

    Args:
        matcher: Um objeto cv2.FlannBasedMatcher configurado para correspondência de características.
        keypoints1: Uma lista de keypoints da primeira imagem.
        descriptors1: Um array NumPy contendo os descritores da primeira imagem.
        keypoints2: Uma lista de keypoints da segunda imagem.
        descriptors2: Um array NumPy contendo os descritores da segunda imagem.
        factor: Um fator de limiar usado para filtrar correspondências ambíguas. Quanto menor o fator, mais rigoroso é o filtro.

    Returns:
        good_matches: Uma lista de correspondências de boa qualidade entre as duas imagens.
        points1: Um array NumPy contendo as coordenadas dos keypoints correspondentes na primeira imagem.
        points2: Um array NumPy contendo as coordenadas dos keypoints correspondentes na segunda imagem.
    """

    # Encontra os dois vizinhos mais próximos para cada descritor na primeira imagem
    matches = matcher.knnMatch(descriptors1, descriptors2, k=2)

    # Inicializa uma lista para armazenar as boas correspondências
    good_matches = []

    # Aplica o teste de razão de Lowe para filtrar correspondências ambíguas
    for m, n in matches:
        if m.distance < factor * n.distance:  # Mantém apenas correspondências onde a distância para o vizinho mais próximo é significativamente menor que a distância para o segundo vizinho mais próximo
            good_matches.append(m)

    # Extrai as coordenadas dos keypoints correspondentes nas duas imagens
    points1 = np.float32([keypoints1[match.queryIdx].pt for match in good_matches]).reshape(-1, 1, 2)
    points2 = np.float32([keypoints2[match.trainIdx].pt for match in good_matches]).reshape(-1, 1, 2)

    return good_matches, points1, points2

In [5]:
def apply_homography(img1, img2, points1, points2):
    """
    Alinha a imagem 'img1' com a imagem 'img2' usando uma transformação de homografia calculada a partir de pontos correspondentes.

    Args:
        img1: A imagem a ser alinhada.
        img2: A imagem de referência para o alinhamento.
        points1: Um array NumPy contendo as coordenadas dos pontos na imagem 'img1'.
        points2: Um array NumPy contendo as coordenadas dos pontos correspondentes na imagem 'img2'.

    Returns:
        aligned_img: A imagem 'img1' alinhada com a imagem 'img2' usando a transformação de homografia.
    """

    # Obtém as dimensões da imagem de referência
    height, width, channels = img2.shape

    # Calcula a matriz de homografia usando RANSAC para robustez contra outliers
    homography, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

    # Aplica a transformação de perspectiva à imagem 'img1' usando a homografia calculada
    aligned_img = cv2.warpPerspective(img1, homography, (width, height))

    return aligned_img

In [6]:
def align_im1_to_im2(img1, img2):
    """
    Alinha a imagem 'img1' com a imagem 'img2' usando correspondência de características e homografia.

    Args:
        img1: A imagem a ser alinhada.
        img2: A imagem de referência para o alinhamento.

    Returns:
        imMatches: Uma imagem mostrando as correspondências encontradas entre as duas imagens.
        aligned_img: A imagem 'img1' alinhada com a imagem 'img2'.
    """

    # Converte as imagens para tons de cinza
    img1Gray = convert_to_grayscale(img1)
    img2Gray = convert_to_grayscale(img2)

    # Calcula keypoints e descritores para ambas as imagens
    keypoints1, descriptors1 = compute_descriptors(img1Gray)
    keypoints2, descriptors2 = compute_descriptors(img2Gray)

    # Cria um objeto matcher para correspondência de características
    matcher = create_matcher(trees=5, checks=50)

    # Encontra correspondências de boa qualidade e suas localizações
    good_matches, points1, points2 = find_good_matches_loc(matcher, keypoints1, descriptors1, keypoints2, descriptors2, factor=0.80)

    # Desenha as correspondências encontradas em uma imagem
    imMatches = cv2.drawMatches(img1, keypoints1, img2, keypoints2, good_matches, None, flags=2)

    # Aplica a homografia para alinhar a imagem 'img2' com a imagem 'img1'
    aligned_img = apply_homography(img2, img1, points2, points1)

    return imMatches, aligned_img

In [7]:


def main(img_path, save_path, match_path):
    """
    Função principal que realiza o alinhamento de imagens em sequência.

    Args:
        img_path: O caminho da pasta contendo as imagens a serem alinhadas.
        save_path: O caminho da pasta onde as imagens alinhadas serão salvas.
        match_path: O caminho da pasta onde as imagens com as correspondências serão salvas

    """

    # Encontra todos os arquivos na pasta de imagens e os ordena naturalmente
    all_files = find_all_files(img_path)
    all_files = natsorted(all_files)
    print(all_files)
    
    aligned_img_list = []
    aligned_img_list.append(read_image(img_path + all_files[0]))
    
    # Itera sobre os arquivos, alinhando cada imagem com a próxima na sequência
    for i in range(len(all_files)-1):
        # Define os caminhos das imagens de origem e destino
        source_img_path = img_path + all_files[i]
        target_img_path = img_path + all_files[i+1]
        #source_img_path = img_path + all_files[0]
        #target_img_path = img_path + all_files[i]

        # Define os nomes dos arquivos de saída para as correspondências e a imagem alinhada
        match_save_as = "matches_" + str(i) + ".jpg" 
        align_save_as = "align_" + str(i) + ".jpg"

        # Lê a imagem de origem
        print("Reading a source image : ", source_img_path)
        source_img = read_image(source_img_path)

        # Lê a imagem de destino
        print("Reading a target image : ", target_img_path);
        target_img = read_image(target_img_path)

        # Alinha as imagens
        print("Aligning images ...")
        #imMatches, aligned_img = align_im1_to_im2(source_img, target_img)
        imMatches, aligned_img = align_im1_to_im2(aligned_img_list[i], target_img)
        
        aligned_img_list.append(aligned_img)
        
        # Salva a imagem com as correspondências
        print("Saving an feature matching image : ", save_path);
        save_image(match_path, match_save_as, imMatches)

        # Salva a imagem alinhada
        print("Saving an aligned image : ", save_path);
        save_image(save_path, align_save_as, aligned_img)

        # Adiciona uma linha em branco para separar a saída de cada iteração
        print("\n")

In [8]:


def main(img_path, save_path, match_path):
    """
    Função principal que realiza o alinhamento de imagens em sequência.

    Args:
        img_path: O caminho da pasta contendo as imagens a serem alinhadas.
        save_path: O caminho da pasta onde as imagens alinhadas serão salvas.
        match_path: O caminho da pasta onde as imagens com as correspondências serão salvas

    """

    # Encontra todos os arquivos na pasta de imagens e os ordena naturalmente
    all_files = find_all_files(img_path)
    all_files = natsorted(all_files)
    print(all_files)
    
    #aligned_img_list = []
    #aligned_img_list.append(read_image(img_path + all_files[0]))

    aligned_img = read_image(img_path + all_files[0])

    # Itera sobre os arquivos, alinhando cada imagem com a próxima na sequência
    for i in range(len(all_files)-1):
        # Define os caminhos das imagens de origem e destino
        source_img_path = img_path + all_files[i]
        target_img_path = img_path + all_files[i+1]
        #source_img_path = img_path + all_files[0]
        #target_img_path = img_path + all_files[i]

        # Define os nomes dos arquivos de saída para as correspondências e a imagem alinhada
        match_save_as = "matches_" + str(i) + ".jpg" 
        align_save_as = "align_" + str(i) + ".jpg"

        # Lê a imagem de origem
        print("Reading a source image : ", source_img_path)
        source_img = read_image(source_img_path)

        # Lê a imagem de destino
        print("Reading a target image : ", target_img_path);
        target_img = read_image(target_img_path)

        # Alinha as imagens
        print("Aligning images ...")
        #imMatches, aligned_img = align_im1_to_im2(source_img, target_img)
        imMatches, aligned_img = align_im1_to_im2(aligned_img, target_img)
        
        #aligned_img_list.append(aligned_img)
        
        # Salva a imagem com as correspondências
        print("Saving an feature matching image : ", save_path);
        save_image(match_path, match_save_as, imMatches)

        # Salva a imagem alinhada
        print("Saving an aligned image : ", save_path);
        save_image(save_path, align_save_as, aligned_img)

        # Adiciona uma linha em branco para separar a saída de cada iteração
        print("\n")

In [12]:
img_path = '/home/lelis/Documents/Projetos/Stereo_Multifocus/depth_from_focus/data/obj_2/input/'
save_path = '/home/lelis/Documents/Projetos/Stereo_Multifocus/depth_from_focus/data/obj_2/output_3/'
match_save_path = '/home/lelis/Documents/Projetos/Stereo_Multifocus/depth_from_focus/data/obj_2/match_save_3/'    

all_files = find_all_files(img_path)
all_files = natsorted(all_files)
print(all_files)

['frame_00000.jpg', 'frame_00001.jpg', 'frame_00002.jpg', 'frame_00003.jpg', 'frame_00004.jpg', 'frame_00005.jpg', 'frame_00006.jpg', 'frame_00007.jpg', 'frame_00008.jpg', 'frame_00009.jpg', 'frame_00010.jpg', 'frame_00011.jpg', 'frame_00012.jpg', 'frame_00013.jpg', 'frame_00014.jpg', 'frame_00015.jpg', 'frame_00016.jpg', 'frame_00017.jpg', 'frame_00018.jpg', 'frame_00019.jpg', 'frame_00020.jpg', 'frame_00021.jpg', 'frame_00022.jpg', 'frame_00023.jpg', 'frame_00024.jpg', 'frame_00025.jpg', 'frame_00026.jpg', 'frame_00027.jpg', 'frame_00028.jpg', 'frame_00029.jpg']


In [13]:
main(img_path, save_path, match_save_path)

['frame_00000.jpg', 'frame_00001.jpg', 'frame_00002.jpg', 'frame_00003.jpg', 'frame_00004.jpg', 'frame_00005.jpg', 'frame_00006.jpg', 'frame_00007.jpg', 'frame_00008.jpg', 'frame_00009.jpg', 'frame_00010.jpg', 'frame_00011.jpg', 'frame_00012.jpg', 'frame_00013.jpg', 'frame_00014.jpg', 'frame_00015.jpg', 'frame_00016.jpg', 'frame_00017.jpg', 'frame_00018.jpg', 'frame_00019.jpg', 'frame_00020.jpg', 'frame_00021.jpg', 'frame_00022.jpg', 'frame_00023.jpg', 'frame_00024.jpg', 'frame_00025.jpg', 'frame_00026.jpg', 'frame_00027.jpg', 'frame_00028.jpg', 'frame_00029.jpg']
Reading a source image :  /home/lelis/Documents/Projetos/Stereo_Multifocus/depth_from_focus/data/obj_2/input/frame_00000.jpg
Reading a target image :  /home/lelis/Documents/Projetos/Stereo_Multifocus/depth_from_focus/data/obj_2/input/frame_00001.jpg
Aligning images ...
keypoints: 39, descriptors: (39, 128)
keypoints: 42, descriptors: (42, 128)
Saving an feature matching image :  /home/lelis/Documents/Projetos/Stereo_Multifocu

# Deph from focus
---

In [14]:
# from pygco import cut_simple
from utils import *
from natsort import natsorted


import argparse
import cv2
import numpy as np

In [ ]:
def focus_stack(aligned_img, gaussian_size, laplacian_size):
    imGray = convert_to_grayscale(aligned_img)
    gaussian_img = cv2.GaussianBlur(imGray, (gaussian_size, gaussian_size), 0)
    laplacian_img = cv2.Laplacian(gaussian_img, cv2.CV_64F, ksize=laplacian_size)
    
    return laplacian_img

In [ ]:
def focus_stack_canny(aligned_img, gaussian_size, canny_threshold1, canny_threshold2):
    # Converte a imagem para escala de cinza
    imGray = convert_to_grayscale(aligned_img)
    
    # Aplica o filtro Gaussiano para suavizar a imagem
    gaussian_img = cv2.GaussianBlur(imGray, (gaussian_size, gaussian_size), 0)
    
    # Aplica a detecção de bordas usando o algoritmo Canny
    canny_img = cv2.Canny(gaussian_img, canny_threshold1, canny_threshold2)
    
    return canny_img

In [ ]:
def focus_measure_cal(cost_volume, kernel_size=9):
    focus_measure = np.zeros_like(cost_volume)
    kernel = np.ones((kernel_size, kernel_size))

    for i in range(len(cost_volume)):
        focus_img = cost_volume[i]
        focus_measure[i] = focus_img*focus_img
        focus_measure[i] = cv2.filter2D(focus_measure[i], -1, kernel)
        
    return focus_measure

In [ ]:
def all_in_focus(img_list, cost_volume, kernel_size, gaussian_size):
    bgr_imgs = np.asarray(img_list)
    
    all_in_focus_img = np.zeros_like(bgr_imgs[0])
    height, width, channels = all_in_focus_img.shape
    
    focus_measure = focus_measure_cal(cost_volume, kernel_size)
    argmax = np.argmax(focus_measure, axis=0)
    
    normalized = 255 - (normalize(argmax) * 255)
    depth_map = cv2.GaussianBlur(normalized, (gaussian_size, gaussian_size), 0)
    
    for i in range(height):
        for j in range(width):
            idx = argmax[i, j]
            all_in_focus_img[i, j, :] = bgr_imgs[idx, i, j, :]
    
    return depth_map, all_in_focus_img

In [ ]:
"""
def graph_cut(cost_volume, unary_scale, pair_scale, n_iter):
    n = len(cost_volume)
    ii, jj = np.meshgrid(range(n), range(n))
    
    unary_cost = normalize(np.stack(cost_volume, axis=-1)) * unary_scale
    pairwise_cost = np.abs(ii - jj) * pair_scale

    graph_img = cut_simple(unary_cost.astype(np.int32), pairwise_cost.astype(np.int32), n_iter)

    return graph_img
"""

In [ ]:
def main(base_path):
    #img_path = base_path + "align/"
    img_path = base_path
    focus_save_path  = base_path + "focus_stack/"
    depth_save_path = base_path + "depth_map/"
    all_focus_save_path = base_path + "all_in_focus/"
    graph_save_path = base_path + "graph_cut/"
    wmf_save_path = base_path + "wmf/"
    save_as = "output.jpg"
    
    img_list = read_images_from_path(img_path)
    print(img_list)
    #img_list = natsorted(img_list)
    stacked_focus_imgs = []
    
    print("Stacking focus using LoG ... ")
    for i, aligned_img in enumerate(img_list):
        focus_save_as = "focus_" + str(i) + ".jpg"
        
        laplacian_img = focus_stack(aligned_img, gaussian_size=5, laplacian_size=5)
        #canny_img = focus_stack_canny(aligned_img, gaussian_size=5, canny_threshold1=10, canny_threshold2=50)

        stacked_focus_imgs.append(laplacian_img)
        #stacked_focus_imgs.append(canny_img)
        
        print("... Saving images ...")
        save_image(focus_save_path, focus_save_as, laplacian_img)
        #save_image(focus_save_path, focus_save_as, canny_img)
        
    cost_volume = np.asarray(stacked_focus_imgs)
    
    print("Extracting focus from each images ...")
    depth_map, all_in_focus_img = all_in_focus(img_list, cost_volume, kernel_size=64, gaussian_size=5)
    print("Saving depth-from-focus image : ", depth_save_path)
    save_image(depth_save_path, save_as, depth_map)
    print("Saving all-in-focus image : ", all_focus_save_path)
    save_image(all_focus_save_path, save_as, all_in_focus_img)
    


In [ ]:
base_path = '/home/lelis/Documents/Projetos/Stereo_Multifocus/depth_from_focus/data/obj_2/output_3/'
main(base_path)